# Creative Testing Significance Testing

### Prerequisites
1. Creative testing data should be in long form (i.e. each row being a respondent and each column being a question)
2. Have the data in either CSV format or gsheets format

### Import libraries

Just run these.

In [1]:
import numpy as np
import pandas as pd
from statsmodels.stats.proportion import proportions_ztest as ztest
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

from authentication.authenticator import Authenticator
from sheets.sheetmanager import SheetManager

In [20]:
csv_in = False
csv_out = False

# TODO: improve accuracy of decimals etc.

results_dp = 2
results_percentage_dp = 2

In [3]:
def get_csv_in(CSV_IN):
    csv_in = CSV_IN
    if csv_in:
        print("Notebook will take in CSV data")
    else:
        print("Notebook will take in gsheets data")
def get_csv_out(CSV_OUT):
    csv_out = CSV_OUT
    if csv_out:
        print("Notebook will save results to a CSV")
    else:
        print("Notebook will update results to a gsheet")

### Options Menu

1. `csv_in`: `True` or `False`. 
<br>True to read in a csv. 
<br>False to read in a gsheet.
<br>--
2. `csv_out`: `True` or `False`. 
<br>True to output results to a csv. 
<br>False to output results to a gsheet.
<br>--
3. `results_dp`: `int`.
<br>Indicate the number of decimal places for non-percentage results
<br>--
4. `results_percentage_dp`: `int`.
<br>Indicate the number of decimal places for percentage results (e.g. Abs Lift %)

In [4]:
interact(get_csv_in, CSV_IN=False)
interact(get_csv_out, CSV_OUT=False)
print()

interactive(children=(Checkbox(value=False, description='CSV_IN'), Output()), _dom_classes=('widget-interact',…

interactive(children=(Checkbox(value=False, description='CSV_OUT'), Output()), _dom_classes=('widget-interact'…

### Initialize necessary strings for gsheets
__Necessary strings__ if either `csv_in` or `csv_out` is `False`.
1. `keys`
<br>--
2. `SCOPES`
<br>--
3. `data_spreadsheetId`
<br>Necessary if `csv_in` is `False`. 
<br>The ID of the raw creative testing data
<br>--
4. `data_data_range`
<br>Necessary if `csv_in` is `False`.<br>Where the data is located (e.g. 'Data' or 'Data!A1:B10')
<br>--
5. `results_spreadsheetId`
<br>Necessary if `csv_out` is `False`. 
<br>The ID of the spreadsheet to output the results to. (Can be the same spreadsheet with the raw data)
<br>--
6. `results_data_range`
<br>Necessary if `csv_out` is `False`. 
<br>Name of sheet/tab to output the results to. 
<br>If tab does not exist, will be created.

In [5]:
keys = 'credentials.json'
SCOPES = ['https://www.googleapis.com/auth/drive']

data_spreadsheetId = '1XpG3wW5CetTDqZy670s641bDvEs60bwhFQAgDEcRkg0'
data_data_range = 'Sheet1'

results_spreadsheetId = '1XpG3wW5CetTDqZy670s641bDvEs60bwhFQAgDEcRkg0'
results_data_range = 'Results'

### Initialize necessary strings for csv
__Necessary strings__ if either `csv_in` or `csv_out` is `True`.

In [7]:
# csv_path = 'test.csv'
# csv_out_path = 'results.csv'

### Authenticate and Initialize Manager to work with Google Sheets
You can run this with no consequence even if you opt to use csv only.
<br> But you __must__ run this if there is gsheet use. There might be a pop-up window asking for authorization. Authorize.

In [6]:
if (not csv_in) or (not csv_out):
    authenticator = Authenticator(keys)
    creds = authenticator.get_creds(SCOPES)
    manager = SheetManager(creds)

### Load in Raw Data as DataFrame
Run this to load in the data.

In [7]:
if not csv_in:
    data_df = manager.get_values(spreadsheetId=data_spreadsheetId,
                            data_range=data_data_range)
else:
    data_df = pd.read_csv(csv_path)

### Define questions here:
1. `group_question`: `str`
<br>Indicate the question/variable that determine's the respondent's group (control or exposed, etc.)
<br>--
2. `control_value`: `str`
<br>Indicate the __value__ in the above question/variable that identifies a respondent to be in the __control__ group
<br>--
3. `weights_variable_name`: `str` or `None`
<br>Indicate the question/variable for the weights. __If no weights variable__, use `None`.


In [8]:
group_question = 'Q1'
control_value = 'control' # string value, even for numbers
weights_variable_name = 'Count' # or None

__Run this if no weights variable:__

In [11]:
if weights_variable_name is None:
    weights_variable_name = 'Count'
    data_df[weights_variable_name] = 1

### Recoding

#### IMPORTANT SECTION

1. `demo_question_list`: `list` (optional)
<br>Indicate demo questions. These questions may be used later on for cuts but will not go through significance testing.
<br> e.g. no use testing Gender and saying that exposure to the ad increase number of Females
2. `desired_dic`: `dict` __(mandatory)__
<br> __PLEASE FOLLOW THE FORMAT__
<br> Create a dictionary indicating the __desired response(s)__ for each __question__
<br> Only questions indicated in this dictionary will be tested and appear in the results.

In [12]:
######
# demo questions
# TO DO: auto cut based on demo in this list
# make it optional
######
demo_question_list = [
    'V or D',
    'Age Group',
]

### Follow the format of
"""
{
 'question_id': [desired option 1, desired option 2(, desired option 3, ...)],
 'question_id': [desired option 1, desired option 2(, desired option 3, ...)]
}
"""

### TODO: take into account "Not Shown" answer options (N/A) answer options

desired_dic = {
    'Q121': [0],
    'Q122.1': [0],
    'Q124.1': [2],
    'Q125': [0],
    'Q126': [0],
    
}


### Recode based on the above dictionary
Just run this. 
<br>Recodes values to `1` (desired) or `0` (not desired) based on the dictionary defined above.

In [10]:
def recode_desired(series):
    series = series.where(series.isin(desired_dic[series.name]), 'X')
    series = series.mask(series.isin(desired_dic[series.name]), 1)
    series[series=='X'] = 0
    series = pd.to_numeric(series)
    return(series)

test_questions = list(desired_dic.keys())

recoded_df = data_df.copy()
recoded_df[test_questions] = recoded_df[test_questions].apply(recode_desired, axis=0)
recoded_df

,Count,Q1,V or D,Format,Site,Smart Home Owner,Q117,Age Group,Q118,Q119,...,Q130.2,Q130.3,Q130.4,Q130.5,Q131.1,Q131.2,Q131.3,Q131.4,Q131.5,Q132
0,1,control,Video,Video (non-CTV),Nine,Yes,23,18-24,0,0,...,2,3,2,7,4,3,4,3,,0
1,1,control,Display,Display,Nine,No,64,55+,0,0,...,2,2,2,2,1,1,1,1,1,2
2,1,control,Display,Display,Nine,No,68,55+,0,0,...,7,7,7,7,,,,,,1
3,1,control,Display,Display,Nine,No,66,55+,0,1,...,7,7,7,7,,,,,,0
4,1,control,Display,Display,News,No,36,25-39,0,0,...,7,7,7,7,,,,,,0
5,1,control,Display,Display,Nine,No,39,25-39,0,0,...,3,3,7,7,2,2,2,,,1
6,1,control,Display,Display,Nine,No,76,55+,0,0,...,6,7,7,7,,5,,,,1
7,1,control,Display,Display,Nine,No,60,55+,0,0,...,6,6,6,5,2,2,2,2,2,1
8,1,control,Display,Display,Nine,No,84,55+,0,0,...,7,7,7,7,,,,,,0
9,1,control,Display,Display,Nine,No,40,40-54,0,1,...,6,7,7,7,2,2,,,,3


### Generate overall cut
Run this to at least get the overall cut.

In [15]:
overall_df = recoded_df.copy()

cuts_dic = {
    'Overall': overall_df,
}

### Additional Cuts (optional)
1. Identify the cuts you want, and save them as Data Frames.
2. Then, add on to the `cuts_dic` they key and the Data Frame.
<br> e.g. `cuts_dic['Male']= male_df`

In [16]:
video = overall_df[overall_df['V or D'] == 'Video']
display = overall_df[overall_df['V or D'] == 'Display']

cuts_dic['Video'] = video
cuts_dic['Display'] = display

cuts_dic.keys()

dict_keys(['Overall', 'Video', 'Display'])

### Execute Tests

Just run this.
Execute the tests on all the cuts indicated above, for the questions that have been selected for testing.

In [21]:
tables = []
control_value = str(control_value)
for cut in cuts_dic.keys():    
    table = cuts_dic[cut].copy()
    table[group_question] = table[group_question].astype(str)
    weights_table = table[[group_question, weights_variable_name]]
    bases = weights_table.groupby(table[group_question]).sum()[weights_variable_name]
    questions = test_questions
    table = pd.pivot_table(table, values=questions, columns=group_question, aggfunc='sum')
    initial_cols = table.columns
    for group in table.columns:
        table[group+' Base'] = bases[group]
        table[group+' Base'] = table[group+' Base'].apply(np.round, decimals=(results_dp))
        table[group+' Desired %'] = table[group] / table[group+' Base']
        table[group+' Desired %'] = table[group+' Desired %'].apply(np.round, decimals=(results_percentage_dp))
        if group != control_value:
            table[group+' Abs Lift %'] = table[group+' Desired %'] - table[control_value+' Desired %']
            table[group+' Abs Lift %'] = table[group+' Abs Lift %'].apply(np.round, decimals=(results_percentage_dp))
            table[group+' p-value'] = table.apply(lambda x: ztest(
                                                                [x[group], x[control_value]],
                                                                [x[group+' Base'], x[control_value+' Base']]
                                                                    )[1], axis=1)
            table[group+' Significance'] = 'No Lift'
            table[group+' Significance'].loc[(table[group+' p-value'] < .2) & (table[group+' Abs Lift %'] > 0)] = 'Directional Lift'
            table[group+' Significance'].loc[(table[group+' p-value'] < .1) & (table[group+' Abs Lift %'] > 0)] = 'Significant Lift'
            table[group+' Significance'].loc[(table[group+' p-value'] < .05) & (table[group+' Abs Lift %'] > 0)] = 'Strong Significant Lift'
            table[group+' Significance'].loc[(table[group+' p-value'] < .1) & (table[group+' Abs Lift %'] < 0)] = 'Negative Lift'

    suffixes = [' Base', ' Desired %', ' Abs Lift %', ' Significance'] # can include control_name+' p-value'
    table['Cut'] = cut
    col_order = ['Cut', control_value, control_value+' Base', control_value+' Desired %']
    for col in initial_cols:
        if col != control_value:
            col_order.append(col)
            for suffix in suffixes:
                col_order.append(col+suffix)
    table = table[col_order]
    #table = table.rename(index=codebook_dic)
    tables.append(table)
    
results = pd.concat(tables, axis=0)

In [18]:
print(results)

Q1          Cut  control  control Base  control Desired %  test  test Base  \
Q121    Overall      188           571               0.33   130        353   
Q122.1  Overall      266           571               0.47   181        353   
Q124.1  Overall      213           571               0.37   168        353   
Q125    Overall      209           571               0.37   160        353   
Q126    Overall      232           571               0.41   179        353   
Q121      Video      143           401               0.36    69        174   
Q122.1    Video      200           401               0.50    93        174   
Q124.1    Video      148           401               0.37    85        174   
Q125      Video      139           401               0.35    78        174   
Q126      Video      163           401               0.41    91        174   
Q121    Display       45           170               0.26    61        179   
Q122.1  Display       66           170               0.39    88 

### Write results automatically to gsheets, or output a csv. 
Based on selected option at the top.

In [22]:
if not csv_out:
    results_to_write = results.reset_index()

    values = []
    values.append(results_to_write.columns.tolist())
    for row in results_to_write.values.tolist():
        values.append(row)

    if results_data_range not in manager.get_existing_sheets_names(spreadsheetId=results_spreadsheetId):
        manager.add_sheet(spreadsheetId=results_spreadsheetId, sheet_name=results_data_range)

    manager.update_values(spreadsheetId=results_spreadsheetId,
                          update_range=results_data_range,
                          values=values)
else:
    results.to_csv(index=True, header=True)